# __Search Image by Image__

- Tutorial Difficulty: ★☆☆☆☆
- 7 min read
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_search/search_image_by_image.ipynb
- References: [Introduction of MNIST DataSet](https://en.wikipedia.org/wiki/MNIST_database), [A Simple Framework for Contrastive Learning of Visual Representations](https://arxiv.org/abs/2002.05709)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Image Vectorization</h4>
    <p>Images(height x width x channel [RGB] x color intensity) are meaningless if the information for each pixel is randomly generated. In other words, an image can only be recognized as an image if each pixel has a specific pattern associated with the surrounding pixels. With this information, it can be inferred that an image can be represented on a low-dimensional feature vector. Recently, studies using machine learning to vectorize and express each image in a low-dimensional space based on similarity have been conducted.</p>
</div>

There are several ways to define the similarity of an image. It can refer to the colors being similar, the objects in the image being similar, or the context of the image being similar (ex. a handwritten number). Although it is difficult to give an exact definition of an image similarity, machine learning learns and vectorizes these general features.

ThanoSQL uses the [Self-Supervised Learning Model](https://en.wikipedia.org/wiki/Self-supervised_learning) to input images into the database and retrieve similar images from it. When you upload images to the ThanoSQL's database, a machine learning algorithm places similar images together while placing non-similar images apart. Image features are derived from an unlabeled dataset and fine-tuned with a small amount of labeled data. Then, it can be used for classification or regression tasks.

Furthermore, ThanoSQL uses machine learning algorithms to vectorize datasets. The vectorized data is stored as a database column in the image table and is used to calculate the similarity(distance).

__The following are use case examples of the ThanoSQL's similar image search algorithm.__

- Input your favorite image and have similar artworks searched for and recommended to you.
- Find similar images within an album containing thousands of photos.
- Store your images in the ThanoSQL's database and create your own search engine or machine learning model utilizing the ThanoSQL Auto-ML regression/classification model.
 
<div class="admonition note">
    <h4 class="admonition-title">In this tutorial</h4>
    <p>👉 This tutorial will use the <mark style="background-color:#FFD79C">MNIST handwriting dataset</mark>. Each image consists of a hand written number between 0 and 9 and is correctly labeled. The MNIST handwriting dataset consists of 1,000 train images and 200 test images.</p>
</div>
    
Create a model that uses ThanoSQL to input handwriting data and retrieves similar images from the database.

[![IMAGE](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png "MNIST data") ](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png)

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET mnist_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY mnist_train 
OPTIONS (overwrite=True)
FROM "thanosql-dataset/mnist_data/mnist_train.csv"

Success


In [4]:
%%thanosql
COPY mnist_test 
OPTIONS (overwrite=True)
FROM "thanosql-dataset/mnist_data/mnist_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"overwrite": determines whether to overwrite a table if it already exists. If set as True, the old table is replaced with the new table (True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. Checking the Dataset__

To create a handwriting classification model, we use the <mark style="background-color:#FFEC92">mnist_train</mark> table from the ThanoSQL workspace [database](https://en.wikipedia.org/wiki/Database). The <mark style="background-color:#FFEC92">mnist_train</mark> table contains the file name, label information, and <mark style="background-color:#FFD79C">MNIST</mark> images' file path. To check the contents of the table, run the query below.

In [5]:
%%thanosql
SELECT * 
FROM mnist_train 
LIMIT 5

,image_path,filename,label
0,thanosql-dataset/mnist_data/train/6782.jpg,6782.jpg,5
1,thanosql-dataset/mnist_data/train/1810.jpg,1810.jpg,5
2,thanosql-dataset/mnist_data/train/33617.jpg,33617.jpg,5
3,thanosql-dataset/mnist_data/train/27802.jpg,27802.jpg,5
4,thanosql-dataset/mnist_data/train/50677.jpg,50677.jpg,5


<div class="admonition note">
    <h4 class="admonition-title">Understanding the Data</h4>
    <p>The <mark style="background-color:#FFEC92">mnist_train</mark> table contains the following information.</p>
    <ul>
        <li><mark style="background-color:#D7D0FF">image_path</mark>: image path</li>
        <li><mark style="background-color:#D7D0FF">filename</mark>: file name</li>
        <li><mark style="background-color:#D7D0FF">label</mark>: image label</li>
    </ul>
</div>

## __2. Create an Image Model__

Create an image vectorization model using the <mark style="background-color:#FFEC92">mnist_train</mark> table referenced in the previous step. Execute the query below to create a model named <mark style="background-color:#E9D7FD">my_image_search_model</mark>.  
(Estimated time required for query execution: 1 min)

In [6]:
%%thanosql
BUILD MODEL my_image_search_model
USING SimCLR
OPTIONS (
    image_col="image_path",
    max_epochs=1,
    overwrite=True
    )
AS 
SELECT * 
FROM mnist_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <mark style="background-color:#E9D7FD">mnist_model</mark>.</li>
        <li>"<strong>USING</strong>" specifies <code>SimCLR</code> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"image_col": the name of the column containing the image path (default: "image_path")</li>
            <li>"max_epochs": number of times to train with the training dataset.</li>
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model. (True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the `mnist_test` images run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column (default: <mark style="background-color:#D7D0FF">convert_result</mark>) in the `mnist_test` table.

In [7]:
%%thanosql
CONVERT USING my_image_search_model
OPTIONS (
    table_name= "mnist_test",
    image_col="image_path",
    column_name="convert_result"
    )
AS 
SELECT * 
FROM mnist_test

,image_path,filename,label,convert_result
0,thanosql-dataset/mnist_data/test/5099.jpg,5099.jpg,6,"[0.6004971, 1.356544, 0.5917841, 1.4965276, 0...."
1,thanosql-dataset/mnist_data/test/9239.jpg,9239.jpg,6,"[0.5917816, 0.5604829, 0.71093637, 0.7439931, ..."
2,thanosql-dataset/mnist_data/test/2242.jpg,2242.jpg,6,"[0.501104, 0.92481977, 0.803071, 1.0276031, 0...."
3,thanosql-dataset/mnist_data/test/3451.jpg,3451.jpg,6,"[0.5152817, 0.86587745, 0.46264905, 0.9430346,..."
4,thanosql-dataset/mnist_data/test/2631.jpg,2631.jpg,6,"[0.7324318, 1.0288409, 0.5339064, 1.3720758, 0..."
...,...,...,...,...
195,thanosql-dataset/mnist_data/test/8045.jpg,8045.jpg,8,"[0.44267064, 0.7129019, 0.5213964, 1.2292978, ..."
196,thanosql-dataset/mnist_data/test/9591.jpg,9591.jpg,8,"[0.2763853, 0.5670233, 0.7088003, 0.7695681, 0..."
197,thanosql-dataset/mnist_data/test/7425.jpg,7425.jpg,8,"[0.40218243, 0.50516707, 0.97546273, 0.9091700..."
198,thanosql-dataset/mnist_data/test/2150.jpg,2150.jpg,8,"[0.76504844, 0.8176137, 0.88861257, 1.7853947,..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <code>my_image_search_model</code> as an algorithm for image vectorizaion.   </li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for image vectorization.
        <ul>
            <li>"table_name": the table name to be stored in the ThanoSQL workspace database. </li>
            <li>"image_col": the name of the column containing the image path. (default: "image_path")</li>
            <li>"column_name": defines the column name that contains the vectorized results. (default: "convert_result")</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Images Using Image Quantization Models__

This step uses the <mark style="background-color:#E9D7FD">my_image_search_model</mark> image vectorization model and the test table to search for images similar to the "923.jpg" image(handwritten 8).

<a href="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png">
    <img alt="IMAGE" src="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png" style="width:100px">
</a>

<p style="text-align:center">923.jpg Image File </p>

In [8]:
%%thanosql
SEARCH IMAGE image='thanosql-dataset/mnist_data/test/923.jpg' 
USING my_image_search_model 
OPTIONS (
    emb_col="convert_result",
    column_name="search_result"
    )
AS
SELECT * 
FROM mnist_test

,image_path,filename,label,convert_result,search_result
0,thanosql-dataset/mnist_data/test/5099.jpg,5099.jpg,6,"[0.6004971, 1.356544, 0.5917841, 1.4965276, 0....",0.969948
1,thanosql-dataset/mnist_data/test/9239.jpg,9239.jpg,6,"[0.5917816, 0.5604829, 0.71093637, 0.7439931, ...",0.949437
2,thanosql-dataset/mnist_data/test/2242.jpg,2242.jpg,6,"[0.501104, 0.92481977, 0.803071, 1.0276031, 0....",0.960603
3,thanosql-dataset/mnist_data/test/3451.jpg,3451.jpg,6,"[0.5152817, 0.86587745, 0.46264905, 0.9430346,...",0.959397
4,thanosql-dataset/mnist_data/test/2631.jpg,2631.jpg,6,"[0.7324318, 1.0288409, 0.5339064, 1.3720758, 0...",0.962196
...,...,...,...,...,...
195,thanosql-dataset/mnist_data/test/8045.jpg,8045.jpg,8,"[0.44267064, 0.7129019, 0.5213964, 1.2292978, ...",0.963523
196,thanosql-dataset/mnist_data/test/9591.jpg,9591.jpg,8,"[0.2763853, 0.5670233, 0.7088003, 0.7695681, 0...",0.951193
197,thanosql-dataset/mnist_data/test/7425.jpg,7425.jpg,8,"[0.40218243, 0.50516707, 0.97546273, 0.9091700...",0.956088
198,thanosql-dataset/mnist_data/test/2150.jpg,2150.jpg,8,"[0.76504844, 0.8176137, 0.88861257, 1.7853947,...",0.968032


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH IMAGE [image|text|audio|video]</strong>" defines the image|text|audio|video file type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for image vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for image searching.
        <ul>
            <li>"emb_col": the column that contains the vectorized results. </li>
            <li>"column_name": defines the name of the column that contains the search results. (default: "search_result")</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, the <code>mnist_test</code> table is used. </li>
    </ul>
</div>

To output the "__SEARCH__" result using the "__PRINT__" clause to output the top four most similar images, run the following query. Though we've only done a minimal amount of training, you can see that images similar to 8 are returned.

In [9]:
%%thanosql
PRINT IMAGE 
AS (
    SELECT image_path, search_result 
    FROM (
        SEARCH IMAGE image='thanosql-dataset/mnist_data/test/923.jpg' 
        USING my_image_search_model 
        OPTIONS (
            emb_col="convert_result",
            column_name="search_result"
            )
        AS 
        SELECT * 
        FROM mnist_test
        )
    ORDER BY search_result DESC 
    LIMIT 4
    )

/home/jovyan/thanosql-dataset/mnist_data/test/923.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/5087.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/7645.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/6618.jpg


<div class="admonition danger">
    <h4 class="admonition-title">Note</h4>
    <p>The training options of the algorithm recognize the image regardless of the image's left-right inversion and color differences. This is because a dog's picture should be recognized as a dog even if it is flipped or has a color difference. If the color feature is important, such as clothing images, or if vertical and horizontal inversions are important, such as numbers, the training options can be changed.</p>
</div>

## __4. In Conclusion__

In this tutorial, we used the `MNIST` handwriting dataset to vectorize images and perform image search. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adding precise tuning and small amounts of labeling to each dataset.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an image similarity search model: contact@smartmind.team</p>
</div>